<!--img src="images\cloudflow.png" width=1400 /-->

# Introduction 


## How to use this notebook
This example runs Sima in locally based on input created in the [Create load case folders with unique Sima input](#loadcase_id) section. The code is only a pilot and only intended for testing. 

This note bookshould be run in the following way:
1. [Installation](#installation) can be run to install all the relevant tools, like python modules and local worker executable
2. [Initialize Workflow](#initialize), run this Python code once at every notebook start to set up basic settings. [Set up custom user parameters](#custom) section should be changed if you want to change workspace or switch between cloud and local run. 2. If you changed something in this section, remember to rerun  [Set up OneWorkflow client](#builder).
3. [Prepare Sima Input](#prepare) sets up reading of the environment input file and build Sima input accordingly. If the input file changes OO needs to change this section.
4. [Run analysis](#run) shall be run each time a new analysis needs to be run.
5. [Run Wasim and Sestra](#runwasim) run Wasim and Sestra using results from above Sima run.


### Hints
If you get this message "#ERROR - Error in WorkerHost: Completed, but failed to download one or more files:", please remember the upload common file step.

# Initialize workflow <a id='initialize'></a>
Run only once when notebook is opened.

## Install  <a id='installation'></a>
Run only when packages and/or local worker  needs to be upgraded. Set _update_packages_ to True first time and when packages should be updated.

In [ ]:
update_packages = False
if update_packages:
    ! pip install --index-url https://test.pypi.org/simple/ dnv-onecompute==0.1.0.527 --force-reinstall
    ! pip install --index-url https://test.pypi.org/simple/ dnv-oneworkflow==0.1.0.527 --force-reinstall
    ! pip install --index-url https://test.pypi.org/simple/ dnv-sesam-commands==0.1.0.527 --force-reinstall
    
    #! pip install --upgrade --index-url https://test.pypi.org/simple/ dnv-onecompute=0.1.0.525
    #! pip install --upgrade --index-url https://test.pypi.org/simple/ dnv-oneworkflow=0.1.0.525
    #! pip install - -upgrade - -index-url https: // test.pypi.org/simple / dnv-sesam-commands = 0.1.0.525
    from oneWorkflowToolBox import *
    await install_workflow_runtime()


## Set up fixed user parameters <a id='fixed'></a>
The below parameters should only be needed to modify once.

In [ ]:
#Sima path must be specified
sima_exe_path = r'C:\Program Files\DNV\Sima V4.4-00'         
tempFolderPath = r"c:\OneWorkflowTemp" #local directory for worker host, should be at root due to issues with long file names on Windows


## Set up custom user parameters <a id='custom'></a>

In [ ]:
# local workspace, all results will be put here after local or cloud runs
workspacePath = r'C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace'
# location of common files for all analysis, has to be below workspacePath
workspaceId = "SE28"
loadcase_file = "test_cases.xlsx"
wasim_input_file = "test_cases_wasim_input.xlsx"
stask_file = "SimaTemplate.stask"
cloud_run = False


## Set up OneWorkflow client <a id='builder'></a>
Run only once workbook is started or if some parameters above are changed.

In [ ]:
from oneWorkflowToolBox import *
workflow_client = one_workflow_client(
    workspaceId, workspacePath, cloud_run, tempFolderPath)#, Platform.Linux, debug=True)
workspace = workflow_client.one_workflow_config.workspace_config
commonfiles_folder = workspace.common_files_directory
results_folder = workspace.results_directory
#If running locally the code below will also start the local workflow host.
if (cloud_run):
    workflow_client.login()


## Upload common files for the job <a id='upload'></a>
This step uploads all common files in folder *commonFilesDirectory*  to the job. Only needed to run if new common files are to be uploaded or workspace changed.

In [ ]:

from dnv.onecompute.directory_client import FileOptions
try:
    workflow_client.upload_common_files(FileOptions(
        # max_size_bytes=124_000,
        #patterns=["**/*.py","**/*.inp"],
        overwrite=True))
except Exception as e:
    print(e)
    print("Ignore this error message if the files are already present.")


## Prepare Sima input  <a id='prepare'></a>


### Create command line input for Sima
The function below will create the command line input that will be used to run Sima. 

In [ ]:
from dnv.onecompute import FileSpecification


def get_commands_inputs(
        stask_file: str,
        case: dict[str, Any])->dict[str, dict[str, Any]]:
    commands = dict[str, Any]()
    commands["--consoleLog"] = ""
    commands["--log-level"] = "ALL"
    commands["--data"] = "."
    commands["--import"] = dict(file=FileSpecification(sharedfolder=True,
                                directory=commonfiles_folder, filename=stask_file))
    commands["--run"] = dict(task="WorkflowTask",
                             workflow="ExampleWorkflow")

    return {"commands": commands, "inputs": case}

### Create load cases with unique Sima input <a id='loadcase_id'></a>
This code generates the input needed for Sima for the individual load cases. The input parameters to vary in the Sima run are give in the loadcasefile.

In [ ]:
import pandas as pd
from dnv.sesam.sima_command import SimaCommand
from dnv.onecompute.flowmodel import ParallelWork
from dnv.oneworkflow import PythonCommand
def get_parallel_work(single_task: bool = False):
    """Returns a parallel processing unit. If the single_task parameter is set to True, the unit will only contain the first task. 
    If set to False, all tasks will be included. The single task option is primarily used for testing purposes."""
    os.chdir(workspace.workspace_path)
    load_cases_parent_folder_name = workspace.load_cases_parent_directory

    parallel_work = ParallelWork()
    parallel_work.work_items.clear()

    # Open environmental input file
    df_cases = pd.read_excel(os.path.join(workspacePath, loadcase_file), index_col=0)
    for loadcase_folder_name, case in df_cases.iterrows():
        load_case_folder = os.path.join(
            load_cases_parent_folder_name, loadcase_folder_name)
        result_folder_lc = os.path.join(workspace.results_directory, loadcase_folder_name)
        # Get SIMA commands and inputs 
        commands_inputs = get_commands_inputs(
             stask_file,  case.to_dict()
        )
        test_command = PythonCommand(
            directory=commonfiles_folder,
            filename="test.py",
            working_dir=loadcase_folder_name,
        )
        # Create SimaCommand instance
        sima_cmd = SimaCommand(sima_exe_path)
        sima_cmd.commands = commands_inputs["commands"]
        sima_cmd.input = commands_inputs["inputs"]
        sima_cmd.sima_result_files =[
                "*-sima.lis",
                "variable*.inp",
                "*.log",
                "results.tda",
                "results.txt",
                "sima_*.res",
                "sys-sima.dat",
                "sima_*.bin",
                "key_sima_*.txt",
                "sima.*"
            ]
        sima_cmd.working_directory = load_case_folder

        # Add work item to ParallelWork instance
        parallel_work.add(sima_cmd, work_unit_id=loadcase_folder_name+"_ID").output_directory(result_folder_lc,
                                                                                         include_files=["**/sima*","**/*.txt", "**/*.lis", "**/*.log"])
        if single_task == True:
            break
    return parallel_work

# Run analysis <a id='run'></a>
This code will fetch data from the blob storage created in the step above, and run all the job tasks. The code will wait for all tasks to complete before downloading the results.

In [ ]:
from oneWorkflowToolBox import run_workflow_async
import json

"""Tests SIMA and Python commands"""
# Upload Input Files
workflow_client.upload_input_files()

# Create Parallel Work Unit and Job
work_unit = get_parallel_work()
job = workflow_client.create_job(work_unit)

job_json = json.dumps(job, default=lambda o: o.encode(), indent=4)
#print(job_json)
# Run workflow
await run_workflow_async(job, workflow_client)


# Run Wasim and Sestra <a id='runwasim'></a>

In [ ]:
from dnv.oneworkflow import PythonCommand, CompositeExecutableCommand
from dnv.sesam.wasim_command import WasimStruCommand,  WasimSetupCommand
from dnv.onecompute.flowmodel import WorkUnit
from SesamUtilities import WasimAndSestraTaskCreator
import pandas as pd
from dnv.oneworkflow import  ParallelWork
from oneWorkflowToolBox import Platform
import json
import os
from dnv.sesam.sestra_command import SestraCommand
workspace.results_directory = "Results"
load_cases = ["test001"]#, "test002", "test003"]

os.chdir(workspacePath)
topSuperElement = 3
# due to 5 field width on Sestra cards we need to use as short name here
additionalTemplateParameters = {
    'FMT': topSuperElement, 'topsel': topSuperElement}

def run_wasim_and_sestra_using_results_from_sima(
        results_directory: str, load_cases: str) -> ParallelWork:
    """Creates a parallel work unit"""
    df_cases = pd.read_excel(os.path.join(
        workspacePath, wasim_input_file), index_col=0)
    parallel_work_units = list[WorkUnit]()

    for casename, case in df_cases.iterrows():
        if not casename in load_cases:
            print("skipping " + casename)
            continue
        
        load_case_result_files_dir = os.path.join(results_directory, casename)
        #For testing
        #python_copy_command = PythonCommand(
        #    directory=commonfiles_folder,
        #    filename="copyfiles.py",
        #    working_dir=load_case_result_files_dir)
        casedict = case.to_dict()
        
        cmd = WasimAndSestraTaskCreator(
            load_case_result_files_dir, commonfiles_folder, casedict, additionalTemplateParameters).CreateTasks()
        work_unit = (
            WorkUnit(cmd, f"post_rerun_{casename}")
            .input_directory(load_case_result_files_dir)
            .output_directory(load_case_result_files_dir, include_files=["**/sima.*", "**/*.txt", "**/*.tda", "**/*.bin", "**/*.log", "**/*.inp", "**/*.lis", "**/*.mlg", "**/*.sin"])
        )
        parallel_work_units.append(work_unit)

    return ParallelWork(parallel_work_units)
print(os.getcwd())
print(workflow_client.workspace_path)

work_unit = run_wasim_and_sestra_using_results_from_sima(
    workspace.results_directory, load_cases)
if not cloud_run:
    workflow_client.upload_common_files()
    workflow_client.upload_result_files()
print(workflow_client.temp_path)


downloadOptions = FileOptions(
    max_size_bytes=11124_000,
    patterns=["**/*.txt", "**/*.lis", "**/*.mlg", "**/*.sin"])
await workflow_client.run_workflow_async(
    work_unit,
    work_item_status_changed=work_item_status_changed_callback(workflow_client, downloadOptions),
    log_job=True
)
